In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import sqlite3
%matplotlib inline

pd.options.display.max_rows = 10

# First Example

In [ ]:
diamonds = pd.read_csv('data/diamonds.csv')
iris = pd.read_csv('data/iris.csv', index_col=0)
titanic = pd.read_csv('data/titanic.csv')
stocks = pd.read_csv('data/stock.csv', index_col=0, parse_dates=['Date'])  # we'll cover this later
stocks

In [ ]:
stocks.plot() 

In [ ]:
stocks.rolling(100, min_periods=5).mean().plot()

In [ ]:
type(stocks)  # it's a DataFrame

In [ ]:
stocks.info()

In [ ]:
stocks.describe()

# Data Structures

The most important data structures in pandas are the `Series` (1-Dimensional) and the `DataFrame` (2-Dimensional).
There are also more dimensional data structures, e.g., `Panel` and `Panel4D` which won't be covered here.

## Series
can be created from any iterable, an index gets created automatically if none is provided

In [ ]:
series = pd.Series([1.0, 4.0, 9.0, 16.0])
series

In [ ]:
series.index  # an int range index is automatically created

In [ ]:
series.values  # the data is kept in a numpy array

In [ ]:
diamonds.values

In [ ]:
type(series.values)

Series can mostly be accessed like numpy arrays:

In [ ]:
series[3]

In [ ]:
series[1:3]

An index can be given and neither the index nor the values need to be numbers:

In [ ]:
capitals = pd.Series(['Paris', 'London', 'Stockholm'], index=['France', 'United Kingdom', 'Sweden'])
capitals

In [ ]:
capitals[0]

In [ ]:
capitals['France']

## DataFrames
`DataFrame` consist of multiple `Series` of the *same length*, sharing a common index.

Can be easily created from a dictionary:

In [ ]:
bands = pd.DataFrame({
    'name': ['The Beatles', 'The Rolling Stones', 'The Who', 'Deep Purple'],
    'from': [1960, 1962, 1964, 1968],
    'to': [1970, np.nan, 1983, 1976],
    'origin': ['Liverpool', 'London', 'London', 'Hertford',], 
    'greatest hit': ['Yesterday', 'Satisfaction', 'My Generation', 'Smoke On the Water'],
})
bands

In [ ]:
bands['from']

In [ ]:
bands.columns

In [ ]:
bands.index

In [ ]:
bands.values  # data is still stored in a numpy array

In [ ]:
bands['sold records']  = [1, 2, 3, 7]
bands

# Common Operations

## getting some information on our data

In [ ]:
bands.info()

In [ ]:
bands

In [ ]:
stocks.count()

In [ ]:
bands.dtypes

In [ ]:
stocks.dtypes

## Setting the index

In [ ]:
bands['name']

In [ ]:
bands = bands.set_index('name')
bands

## Mostly behaves like numpy arrays

In [ ]:
stocks

In [ ]:
stocks * 10

In [ ]:
stocks - 8

obviously only were it makes sense

In [ ]:
try:
    bands / 100
except TypeError as error:
    print(error)

In [ ]:
bands.max() # or not?

In [ ]:
bands.median()  # for some operations only numeric values are included

In [ ]:
stocks.min()

In [ ]:
stocks.idxmin()

In [ ]:
stocks.min(axis=0)  # axis=0 is the default

<div class="alert alert-success">
    <b>EXERCISE:</b> Check the maximum values and the dates when the maximum values of the `stocks` occurred
</div>

In [ ]:
stocks.max()

In [ ]:
stocks.iloc[np.where(stocks['AAPL'] == 133.00)[0]]

## adding new data
* add new data by assigning it like to a dictionary
* there is also an assign methods to basically do the same
* rename columns with `.rename()`

In [ ]:
bands['years active'] = bands['to'] - bands['from']
bands

<div class="alert alert-success">
    <b>EXERCISE:</b> Add a `volume` (x \* y \* z) column to the diamonds DataFrame and test if carat is the same as the volume
</div>

In [ ]:
diamonds.head()

In [ ]:
diamonds['volume'] = diamonds['x'] * diamonds['y'] * diamonds['z']
diamonds['volume'] / diamonds['carat']

<div class="alert alert-success">
    <b>EXERCISE:</b> Add a absolute price changes columns to stocks
</div>

In [ ]:
diffs = stocks.diff()
stocksdiff = stocks.copy()
for symbol in diffs.columns:
    stocksdiff[symbol + ' diff'] = diffs[symbol]
stocksdiff

In [ ]:
for symbol in stocks:
    stocksdiff[symbol + ' rel'] = stocksdiff[symbol + ' diff'] / stocks[symbol]
stocksdiff

In [ ]:
stocks

In [ ]:
stocks.rename(columns={'AAPL': 'Apple', 'AMZN': 'Amazon'})

# Selecting Data

## Selecting Columns

There are several ways to get columns from a DataFrame:

* The same way you would get an dictionary element
* as an attribute (this only works if the column name is a valid python identifier, stocks.1 wouldn't work)

In [ ]:
stocks['AAPL']

In [ ]:
stocks.1AMZN

You can select multiple columns by using a list:

In [ ]:
stocks[['MSFT', 'AAPL']]

## Selecting Rows
* Use indices or numbers to select from Series
* Use slices to select rows from DataFrames (accessing rows like this `stocks[150]` doesn't work)

In [ ]:
capitals[0]

In [ ]:
capitals['France']

In [ ]:
capitals

In [ ]:
import datetime as dt

In [ ]:
stocks['2010-10-11':'2010-10-12']

In [ ]:
stocks[:5]

In [ ]:
stocks[-5:]

In [ ]:
stocks[::2]  # every second entry

Or slicing:
<div class="alert alert-warning">
<b>Careful, the end is included!</b>

In [ ]:
stocks[0:]

Or index via bool sequences (of the right length)

In [ ]:
stocks['AAPL'] > 90

In [ ]:
stocks[(stocks['AAPL'] > 90).values]

In [ ]:
diamonds

In [ ]:
diamonds[diamonds.color.isin(['E', 'F', 'G'])]

In [ ]:
df = diamonds[['color', 'cut']]

In [ ]:
df

## Select by label with .loc

`.loc` allows selecting and slicing by label and lists of labels

<div class="alert alert-warning">
<b>Careful: the end is included!</b>

In [ ]:
stocks.loc[dt.date(2012, 8, 7)]

In [ ]:
stocks.loc[dt.date(2012, 8, 8):dt.date(2012, 9, 7)]

In [ ]:
stocks.loc[:, 'AAPL']  # needs to be in the right dimension stocks.loc['AAPL'] doesn't work

In [ ]:
stocks.loc[[dt.date(2010, 10, 1), dt.date(2010, 11, 1), dt.date(2010, 12, 1)], ['MSFT', 'AAPL', 'AMZN']] 

## Select by index with .iloc

Select and slice by index and lists of indices

In [ ]:
stocks.iloc[0:2]

In [ ]:
stocks

In [ ]:
stocks.iloc[:, 2]

In [ ]:
stocks.iloc[[1, 10, 100, 1000], [2, 0, 1]]

<div class="alert alert-success">
        <b>EXERCISE:</b> 
        <li>access the "Fare" and the "class" column</li>
        <li>check how much the passengers in First class paid on average</li>
</div>

In [ ]:
titanic = pd.read_csv('data/titanic.csv', index_col=0)
titanic.head()

In [ ]:
titanic[titanic['class'] == 'First']['fare'].mean()

## Summary

* .name allows quick access to columns
* `[]` allows access to columns or rows
* .loc for label based access
* .iloc for index based acces 

# Plotting

In [ ]:
stocks.plot()

There are several different kinds of plots available, which take different keywords.

If in doubt, consult the [documentation](https://pandas.pydata.org/pandas-docs/stable/visualization.html).


In [ ]:

diamonds['price'].plot(kind='hist', bins=20);


In [ ]:
pd.Series([3, 5, -1, 8]).plot(kind='bar')

In [ ]:
pd.DataFrame({'a': [1, 4, 8], 'b': [3, 1, 1], 'c': [1, 1, 1]}).plot(kind='barh')

In [ ]:
pd.DataFrame({'a': [1, 4, 8], 'b': [3, 1, 1], 'c': [1, 1, 1]}).plot(kind='barh', stacked=True)

multiple calls to plot() in one cell create a single figure

In [ ]:
pd.Series([1, 8, 7, 3, 110, 12, -3, 16, 21, -30]).plot(kind='kde')
pd.Series([11, 84, 3, 14, 45, ]).plot(kind='kde')

In [ ]:
diamonds.iloc[:5].plot(kind='scatter', x='carat', y='x')

In [ ]:
diamonds.loc[3, 'x'] = np.nan

In [ ]:
pd.Series([3, 5, 1, 8]).plot(kind='pie')  # please don't use those

and many more...
<div class="alert alert-success">
    <b>EXERCISE:</b> Scatter-plot AAPL and MSFT
</div>

In [ ]:
stocks.plot(kind='scatter', x='AAPL', y='MSFT')

<div class="alert alert-success">
    <b>EXERCISE:</b>plot histograms of the stock diffs (absolute and relative)
</div>

In [ ]:
stocksdiff[['AMZN diff', 'AAPL diff', 'MSFT diff']].plot(kind='hist', bins=200)

In [ ]:
stocksdiff[['AMZN rel', 'AAPL rel', 'MSFT rel']].plot(kind='hist', bins=200)

<div class="alert alert-success">
    <b>EXERCISE:</b> Scatter-plot diamond carat vs depth and use the price as the size (argument `s`)
</div>

In [ ]:
diamonds.plot(kind='scatter', x='carat', y='depth', s=diamonds['price'] / 100)

<div class="alert alert-success">
    <b>EXERCISE:</b> Plot a labelled histograms of the iris' petal width, seperated by species
</div>

In [ ]:
for species, group in iris.groupby('species'):
    group['petal_width'].hist(alpha=0.5, label=species)
plt.legend();

# Groupby
* allows for grouping by certain properties
* often used for "split-apply-combine" workflow
* mostly useful for categorical data

In [ ]:
titanic.head()

In [ ]:
titanic.groupby('class').aggregate(np.mean)

In [ ]:
fig, axx = plt.subplots(1)
for num, (class_, data) in enumerate(titanic.groupby('class')):
    colors = {0: 'red', 1: 'blue', 2: 'green'}
    data.plot(x='age', y='fare', kind='scatter', label=class_, ax=axx, color=colors[num])

In [ ]:
titanic.groupby('class')[['fare', 'age']].aggregate(np.mean).plot(kind='bar')

In [ ]:
titanic.groupby('class')

In [ ]:
titanic.groupby('class')['fare'].aggregate(np.mean)

<div class="alert alert-success">
    <b>EXERCISE:</b>
    <li>See if there are differences in the survival rates between male and females (you can also use statistical tests)</li>
    <li>Did people who survive pay a higher fare?</li>
    <li>Try what happens when you groupby a tuple of columns</li>
</div>

In [ ]:
titanic.groupby('survived').aggregate(np.mean)

In [ ]:
titanic.groupby(['survived', 'class']).aggregate(np.mean)

<div class="alert alert-success">
    <b>EXERCISE:</b>
    <li>Group the iris data set by species and plot petal vs sepal length and petal length vs width
</div>

In [ ]:
fig, axx = plt.subplots(1, 2, figsize=(12, 4))
for num, (species, data) in enumerate(iris.groupby('species')):
    data.plot(x='petal_length', y='petal_width', kind='scatter',ax=axx[0], color='C{}'.format(num), label=species)
    data.plot(x='petal_length', y='sepal_length', kind='scatter',ax=axx[1], color='C{}'.format(num), label=species)

* `TimeGrouper` lets you group timeindexed data by time

In [ ]:
stocks.groupby([pd.TimeGrouper(freq='10D')]).aggregate(np.mean).plot()

In [ ]:
stocks['weekday'] = stocks.index.weekday_name
stocks.head()

In [ ]:
x = stocks.copy()

In [ ]:
titanic

In [ ]:
sss = titanic[titanic['age'] > 20]
sss.loc[:, 'fare'] = 99
sss

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 4))
groupby = stocks[['MSFT', 'weekday']].groupby('weekday')
groupby.boxplot(subplots=False)

# Merging Data
* `.concat()` allows for stacking DataFrames along any axis
* `.merge()` allows more compilated merging, set argument `how` to one of `inner`, `outer`, `left` or `right`

In [ ]:
names1 = pd.DataFrame({
    'id': [1, 4, 3, 2, 7, 8],
    'first name': ['Alexander', 'Brooklyn', 'Christopher', 'Destiny', 'Ethan', 'Faith'],
    'last name': ['Abner', 'Bach', 'Carlisle', 'Dawson', 'Egger', 'Fairbanks'],
})

names2 = pd.DataFrame({
    'id': [5, 6, 9, 10],
    'first name': ['George', 'Holly', 'Alexander', 'Brooklyn'],
    'last name': ['Garret', 'Hansen', 'Abner', 'Bach'],
})

scores_a = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'score': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
})

scores_b = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 3, 4],
    'score': [5, 10, 15, 20, 25, 30, 40, 50, 55, 60, 65, 70],
})
pd.options.display.max_rows = 20

In [ ]:
names = pd.concat([names1, names2])
names

In [ ]:
pd.concat([names, names2], axis=0)

In [ ]:
pd.concat([scores_a, scores_b], axis=1, )  # NaN gets inserted for missing values

In [ ]:
pd.merge(names, scores_a, on='id')

In [ ]:
pd.merge(names, scores_b, on='id')

In [ ]:
pd.merge(names1, names2, on=['first name', 'last name'])

In [ ]:
pd.merge(names1, names2, on=['first name'])

In [ ]:
names1.merge(names2, on='first name')

In [ ]:
pd.merge(names1, names2, on=['first name', 'last name'], how='inner')

In [ ]:
pd.merge(names1, names2, on=['first name'], how='outer')

In [ ]:
pd.merge(names1, names2, on='id', how='left')

In [ ]:
pd.merge(scores_a, scores_b, on='id', how='outer')

In [ ]:
scores = pd.merge(scores_a, scores_b, on='id', how='right', suffixes=('_reaction', '_recall'))
scores

In [ ]:
scores.drop_duplicates(['id'], keep=False)

# IO
https://pandas.pydata.org/pandas-docs/stable/io.html

pandas has build in support for various data formats, among others: CSV, Excel Files, HDF5, SQL

It's mostly as easy as `pd.read_csv('path/to/csv', some=arguments)` and `pd.to_csv('path/to/csv')`

## CSV

In [ ]:
df = pd.read_csv('data/iris2.csv')
df.columns = [1, 2, 3, 4, '5', '5']
df['5']

In [ ]:
float('nan')

pandas doesn't know that the first column should be an index

In [ ]:
# most of the arguments are unnecessary for this file, but you get the idea
df = pd.read_csv('data/stock.csv', index_col=0, parse_dates=['Date'], sep=',', encoding='ascii')
df.head()

Let's save it in another CSV, but tab seperated

In [ ]:
df.to_csv('data/stock_out.csv', sep='\t', )

In [ ]:
!head data/stock_out.csv

## SQL
* the pd.read_sql_query(), pd.read_sql(), and pd.to_sql() let you read and write sql
* need an SQLAlchemy engine or SQLite connection

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("data/titanic.db")

titanic.to_sql('titanic', conn, if_exists='replace', )
conn.close()

In [ ]:
conn = sqlite3.connect("data/titanic.db")
cur = conn.cursor()
cur.execute("select * from titanic limit 5;")
print(cur.fetchall())
cur.close()

In [ ]:
pd.read_sql_query('SELECT fare, class FROM titanic WHERE survived == 0;', conn)

# Time series
* pandas can easily parse data time for us
* see `pd.to_datetime()`'s documentation on how to specify the format (it often does the right thing anyway)
* many read methods support reading datetime strings
* DataTimeIndex has lots of properties to show days, hours, years, weekdays, etc.
* data with DateTimeIndex gets properly labeled X-Axis

In [ ]:
!head data/flowdata.csv

In [ ]:
pd.read_csv("data/flowdata.csv")

In [ ]:
flow = pd.read_csv("data/flowdata.csv", index_col=0, parse_dates=True)
flow

In [ ]:
flow.index

In [ ]:
flow.index.day

In [ ]:
flow.index.weekday

In [ ]:
flow.plot()

## Selecting data by date
* can select rows by datetime

In [ ]:
import datetime as dt
flow[dt.datetime(2011, 1, 1):dt.datetime(2011, 2, 1)]

In [ ]:
flow[pd.to_datetime('2011-1-1 10:00'):pd.to_datetime('2011-1-2 14:00')]

In [ ]:
flow['2011-1-1 10:00':'2011-1-2 14:00']

In [ ]:
flow['2011']

In [ ]:
flow['2011-6']

<div class="alert alert-success">
    <b>EXERCISES:</b>

<li>Select all data from all Januaries
<li>Select all data from 2010 to 2012
<li>Select all data from the weekend
<li>Select all data between 9:00 and 17:00 on every day
</div>

In [ ]:
flow[flow.index.month == 1]

In [ ]:
flow['2010':'2012']

In [ ]:
flow[flow.index.weekday.isin([6, 7])]

In [ ]:
flow[(flow.index.hour > 9) & (flow.index.hour < 17)]

## Resampling
* often data isn't sampled as needed
* `.resample('10D').mean()` resamples to every ten days and uses the mean for resampling
* shortcuts:
* ```
B 	business day frequency
D 	calendar day frequency
W 	weekly frequency
M 	month end frequency
A 	year end frequency
H 	hourly frequency
T, min 	minutely frequency
S 	secondly frequency```

there are many more

* need to specify how to resample
* for downsampling: `.mean()`, `.sum()`, `.max()`, ...
* for upsampling: `.pad()`, `bfill()`, ...

* or apply custom sampler with `.apply()`

In [ ]:
flow.resample('D').mean()

In [ ]:
flow.resample('M').mean().plot();

In [ ]:
flow.resample('H').asfreq().head(10)

In [ ]:
flow.resample('H').pad().head()

In [ ]:
x = flow.resample('H')
x

In [ ]:
def custom_resampler(array_like):
    return np.sum(array_like) * 5

flow.resample('H').apply(custom_resampler)

<div class="alert alert-success">
    <b>EXERCISES:</b>
<li>Plot minimum and maximum value from each month for L06_347
<li>Resample the flow data set to a weekly sample rate and write a custom resampler that returns the difference between the maximum and minimum in the week

</div>

In [ ]:
fig, ax = plt.subplots(1)
flow['L06_347'].resample('1M').min().plot(ax=ax, label='min')
flow['L06_347'].resample('1M').max().plot(ax=ax, label='max')
ax.legend();

In [ ]:
def dif(array):
    return np.abs(array.max() - array.min())

flow.resample('1W').apply(dif).plot();


# Reshaping Data
## pivot
* use `pivot_table()` for pivoting your DataFrames to make them broader
* there is also `.pivot()` which works like excels pivot

In [ ]:
titanic.head()

In [ ]:
titanic.pivot_table(index='sex', columns='survived', values='fare')  # default is taking the mean

In [ ]:
df = titanic.pivot_table(index='sex', columns=['survived', 'class', 'embarked'], values='fare', aggfunc=len)

In [ ]:
df

In [ ]:
type(df[0])

<div class="alert alert-success">
    <b>EXERCISES:</b>
<li>Create a table sex vs class, containing the average survival rates
<li>Create a table sex vs class and embark town, contaning the average fares

</div>

In [ ]:
titanic.pivot_table(index='class', columns='sex', values='survived', aggfunc='mean')

In [ ]:
titanic.pivot_table(index=['class', 'embarked'], columns='sex', values='fare', aggfunc='mean')

## Melt
* `.melt()` unpivots your DataFrames to make them longer (and "tidy")

In [ ]:
titanic_p = titanic.pivot_table(index='sex', columns='class', values='fare').reset_index()
titanic_p.columns.name = None
titanic_p

In [ ]:
pd.melt(titanic_p)  # this isn't what we want yet

In [ ]:
pd.melt(titanic_p, id_vars='sex')

* you can also use `.stack()` and `.unstack()` for basically the same functionality as pivot and melt

# TODO

* .query and .eval